In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [8]:
from sklearn.model_selection import train_test_split

In [2]:
from cosmoNODE.loaders import LC

In [3]:
from cosmoNODE import utils

In [4]:
# data formatted according to https://github.com/YuliaRubanova/latent_ode/issues/2
# (record_id, observation_times, values, mask, labels)
# working on parse_dataset
fluxnet = utils.FluxNet()
curve = fluxnet.curves[0]

In [9]:
train_data, test_data = train_test_split(fluxnet.curves, train_size=0.8, random_state=42, shuffle=True)


In [ ]:
record_id, tt, vals, mask, labels = train_data[0]